<a href="https://colab.research.google.com/github/Eamonmca/sc_generative_demo/blob/master/Wasserstein_GAN_GP_Full_Evaluation_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 %matplotlib inline
!python -m pip uninstall matplotlib -Y
!pip install matplotlib==3.1.3 
!pip install wandb
!pip install scanpy
import matplotlib.pyplot as plt
import tempfile




Usage:   
  /usr/bin/python3 -m pip uninstall [options] <package> ...
  /usr/bin/python3 -m pip uninstall [options] -r <requirements file> ...

no such option: -Y
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.3-cp39-cp39-linux_x86_64.whl
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scanpy 1.9.3 requires matplotlib>=3.4, but you have matplotlib 3.1.3 which is incompatible.
plotnine 0.10.1 requires matplotlib>=3.5.0, but you have matplotlib 3.1.3 which is incompatible.
mizani 0.8.1 requires matplotlib>=3.5.0, but you have matplotlib 3.1.3 which is incompatible.
arviz 0.15.1 requires matplotlib>=3.2, but you have 

In [2]:
import torch
from torch import nn, optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
import torchvision
from torchvision import datasets, transforms
import numpy as np  # Thinly-wrapped numpy
import torch.autograd as autograd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import scanpy as sc
import plotly.express as px
import plotly.io as pio
import sklearn.preprocessing
import sklearn.model_selection
from tqdm import tqdm
from scipy import sparse
import os
from datetime import datetime




torch.autograd.set_detect_anomaly(True)

In [3]:
import platform

def get_device_and_gmount():
    # Get the operating system and version
    os = platform.system()
    version = platform.release()

    # Get the machine's architecture
    arch = platform.machine()

    # Set the default renderer based on the operating system
    if os == 'Darwin':
        pio.renderers.default = 'notebook'
        print("Using Apple MPS on Macbook Pro")
    
    elif os == 'Linux':
        pio.renderers.default = 'colab'
        print("Using Colab on Linux")

    # Set the device based on the machine's architecture
    if arch == 'x86_64':
        device = torch.device('mps') if os == 'Darwin' else torch.device('cuda')
        gmount = True if os == 'Linux' else False
    else:
        device = torch.device('cpu')
        gmount = False

    print("Using device:", device)
    
    return device, gmount


In [4]:
device, gmount = get_device_and_gmount()


Using Colab on Linux
Using device: cuda


In [5]:
if gmount:
    from google.colab import drive
    drive.mount('/content/drive')
    scdata = sc.read_h5ad("/content/drive/MyDrive/scintegration/GEX.h5ad")
else:
    scdata = sc.read_h5ad("/Users/eamonmcandrew/Desktop/Single_cell_integration/Data/Multi-ome/GEX.h5ad")

Mounted at /content/drive


In [6]:
scdata

AnnData object with n_obs × n_vars = 69249 × 13431
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [7]:
def stratified_split(data, test_size, random_state, split_criteria):
    """
    Splits the data into train and test sets stratified by the batch column
    """
    train = []
    test = []
    for batch in data.obs[split_criteria].unique():
        batch_data = data[data.obs[split_criteria] == batch]
        batch_train, batch_test = sklearn.model_selection.train_test_split(batch_data, test_size=test_size, random_state=random_state)
        batch_train, batch_test = list(batch_train.obs.index), list(batch_test.obs.index)
        train.extend(batch_train)
        test.extend(batch_test)
        
    return train, test


In [8]:
if gmount == True:
    from google.colab import drive
    drive.mount('/content/drive')
    scdata = sc.read_h5ad("/content/drive/MyDrive/scintegration/GEX.h5ad")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import wandb
wandb.login()


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [10]:
import torch
import numpy as np
import sklearn.preprocessing

class GEX_Dataset(torch.utils.data.Dataset):
    def __init__(self, data, scaler=None, cat_var=None, label_encoder=None):
        self.data = data
        self.values = np.asarray(data.X.todense())
        self.cat_var = cat_var

        label_encoder_functions = {
            "numeric": lambda: torch.tensor(sklearn.preprocessing.LabelEncoder().fit_transform(self.data.obs[self.cat_var]), dtype=torch.long),
            "range_map": lambda: sklearn.preprocessing.LabelEncoder().fit_transform(self.data.obs[self.cat_var]).reshape(-1, 1),
            "one_hot": lambda: sklearn.preprocessing.OneHotEncoder().fit_transform(sklearn.preprocessing.LabelEncoder().fit_transform(self.data.obs[self.cat_var]).reshape(-1, 1)).toarray()
        }

        if label_encoder in label_encoder_functions:
            cat_var_data = label_encoder_functions[label_encoder]()
            if label_encoder == "range_map":
                cat_var_data = torch.tensor(sklearn.preprocessing.MinMaxScaler().fit_transform(cat_var_data), dtype=torch.float32)
            elif label_encoder == "one_hot":
                cat_var_data = torch.tensor(cat_var_data, dtype=torch.float32)
            self.cat_var_data = cat_var_data
        else:
            self.cat_var_data = None

        scaler_functions = {
            "Standard": lambda: (sklearn.preprocessing.StandardScaler().fit(self.values), sklearn.preprocessing.StandardScaler().fit_transform(self.values)),
            "MinMax": lambda: (sklearn.preprocessing.MinMaxScaler().fit(self.values), sklearn.preprocessing.MinMaxScaler().fit_transform(self.values))
        }

        if scaler in scaler_functions:
            self.scaler_obj, scaled_values = scaler_functions[scaler]()
            self.scaled_values = torch.tensor(scaled_values, dtype=torch.float32)
        else:
            self.scaled_values = torch.tensor(self.values, dtype=torch.float32)
            self.scaler_obj = None

    @property
    def n_features(self):
        return self.values.shape[1]

    @property
    def n_catagories(self):
        return self.cat_var_data.shape[1] if self.cat_var_data is not None else 0

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.scaled_values[idx], self.cat_var_data[idx]

    def inverse_transform(self, scaled_data):
        if self.scaler_obj is not None:
            data = self.scaler_obj.inverse_transform(scaled_data)
            data = np.asarray(data)
        else:
            data = np.asarray(scaled_data)
        return data


In [11]:
def stratified_split(data, test_size, random_state, split_criteria):
    """
    Splits the data into train and test sets stratified by the batch column
    """
    train = []
    test = []
    for batch in data.obs[split_criteria].unique():
        batch_data = data[data.obs[split_criteria] == batch]
        batch_train, batch_test = sklearn.model_selection.train_test_split(batch_data, test_size=test_size, random_state=random_state)
        batch_train, batch_test = list(batch_train.obs.index), list(batch_test.obs.index)
        train.extend(batch_train)
        test.extend(batch_test)
        
    return train, test


In [12]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, input_size, dropout, hidden_sizes, output_size, use_batch_norm):
        super().__init__()

        self.input_size = input_size
        self.dropout = dropout
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.use_batch_norm = use_batch_norm

        # create a list of layers
        layers = []

        # input layer
        layers.append(nn.Linear(self.input_size, self.hidden_sizes[0]))
        if self.use_batch_norm:
            layers.append(nn.BatchNorm1d(self.hidden_sizes[0]))
        layers.append(nn.ReLU())
        if self.dropout > 0:
            layers.append(nn.Dropout(p=self.dropout))

        # hidden layers
        for i in range(1, len(self.hidden_sizes)):
            layers.append(nn.Linear(self.hidden_sizes[i-1], self.hidden_sizes[i]))
            if self.use_batch_norm:
                layers.append(nn.BatchNorm1d(self.hidden_sizes[i]))
            layers.append(nn.ReLU())
            if self.dropout > 0:
                layers.append(nn.Dropout(p=self.dropout))

        # output layer
        layers.append(nn.Linear(self.hidden_sizes[-1], self.output_size))
        

        # create the model using Sequential
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [13]:
class Discriminator(nn.Module):
  def __init__(self, input_size, dropout, hidden_sizes, output_size, use_batch_norm):
    super().__init__()

    self.input_size = input_size
    self.dropout = dropout
    self.hidden_sizes = hidden_sizes
    self.output_size = output_size
    self.use_batch_norm = use_batch_norm

    # create a list of layers
    layers = []

    # input layer
    layers.append(nn.Linear(self.input_size, self.hidden_sizes[0]))
    layers.append(nn.LeakyReLU(0.2))
    if self.dropout > 0:
      layers.append(nn.Dropout(p=self.dropout))

    # hidden layers
    for i in range(1, len(self.hidden_sizes)):
      layers.append(nn.Linear(self.hidden_sizes[i-1], self.hidden_sizes[i]))
      if self.use_batch_norm:
        layers.append(nn.InstanceNorm1d(self.hidden_sizes[i]))
      layers.append(nn.LeakyReLU(0.2))
      if self.dropout > 0:
        layers.append(nn.Dropout(p=self.dropout))

    # output layer
    layers.append(nn.Linear(self.hidden_sizes[-1], self.output_size))

    # create the model using Sequential
    self.model = nn.Sequential(*layers)

  def forward(self, x):
    return self.model(x)



In [14]:
class critic(nn.Module):
  def __init__(self, input_size, dropout, hidden_sizes, output_size, use_batch_norm):
    super().__init__()

    self.input_size = input_size
    self.dropout = dropout
    self.hidden_sizes = hidden_sizes
    self.output_size = output_size
    self.use_batch_norm = use_batch_norm

    # create a list of layers
    layers = []

    # input layer
    layers.append(nn.Linear(self.input_size, self.hidden_sizes[0]))
    layers.append(nn.LeakyReLU(0.2))
    if self.dropout > 0:
      layers.append(nn.Dropout(p=self.dropout))

    # hidden layers
    for i in range(1, len(self.hidden_sizes)):
      layers.append(nn.Linear(self.hidden_sizes[i-1], self.hidden_sizes[i]))
      if self.use_batch_norm:
        layers.append(nn.nn.LayerNorm(self.hidden_sizes[i]))
      layers.append(nn.LeakyReLU(0.2))
      if self.dropout > 0:
        layers.append(nn.Dropout(p=self.dropout))

    # output layer
    layers.append(nn.Linear(self.hidden_sizes[-1], self.output_size))

    # create the model using Sequential
    self.model = nn.Sequential(*layers)

  def forward(self, x):
    return self.model(x)

In [15]:
# Define the GAN model
class GAN(nn.Module):
    def __init__(self, generator, discriminator):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator

    def forward(self, x):
        return self.discriminator(self.generator(x))

In [16]:
def get_noise(n_samples, z_dim, device=device):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim),
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    '''
    return torch.randn(n_samples,z_dim,device=device)

In [17]:
def generate_fake_data_and_combine(original_data, generator, z_dim, batch_size):
    # Store original batch and cell_type information
    original_batch_info = original_data.obs['batch'].copy()
    original_cell_type_info = original_data.obs['cell_type'].copy()
    
    generator.eval()
    noise = get_noise(batch_size, z_dim, device=device)
    fake_data = generator(noise)
    fake_adata = sc.AnnData(fake_data.detach().cpu().numpy())

    # add an index column to the fake data, starting at the number of real cells
    fake_adata.var.index = original_data.var.index
    adata = sc.AnnData.concatenate(original_data, fake_adata, batch_key='group', batch_categories=['real', 'fake'])

    # Reassign original batch and cell_type information
    adata.obs.loc[adata.obs['group'] == 'real', 'batch'] = original_batch_info.values
    adata.obs.loc[adata.obs['group'] == 'real', 'cell_type'] = original_cell_type_info.values

    # Convert 'batch' and 'cell_type' columns to categorical dtype if not already
    if not pd.api.types.is_categorical_dtype(adata.obs['batch']):
        adata.obs['batch'] = adata.obs['batch'].astype('category')
    if not pd.api.types.is_categorical_dtype(adata.obs['cell_type']):
        adata.obs['cell_type'] = adata.obs['cell_type'].astype('category')

    # Add "fake" as a category for 'cell_type' column
    adata.obs['cell_type'] = adata.obs['cell_type'].cat.add_categories('fake')

    # set the "cell_type" columns for the fake cells to "fake"
    adata.obs.loc[adata.obs['group'] == 'fake', 'cell_type'] = 'fake'

       # Add "fake" as a category for 'cell_type' column
    adata.obs['batch'] = adata.obs['batch'].cat.add_categories('fake')

    # set the "cell_type" columns for the fake cells to "fake"
    adata.obs.loc[adata.obs['group'] == 'fake', 'batch'] = 'fake'

    # compute the UMAP of the combined data
    sc.pp.neighbors(adata, n_neighbors=10)
    sc.tl.umap(adata)

    return adata


In [18]:
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns


def umap_with_highlighted_category(adata, category_column, highlight_category, highlight_color=(0, 0, 1, 1)):
    # Get unique categories from the specified column
    unique_categories = adata.obs[category_column].cat.categories.tolist()

    # Add the 'fake' category to the unique_categories list
    unique_categories.append(highlight_category)

    # Create a custom color map for the unique categories, using the 'viridis' colormap
    cmap = plt.cm.get_cmap('viridis', len(unique_categories))

    # Prepare color map dictionary, where key is category, and value is color
    cmap_dict = {cat: cmap(i) for i, cat in enumerate(unique_categories)}

    # Change the color of the highlight category to the specified highlight color
    cmap_dict[highlight_category] = highlight_color

    return cmap_dict



In [19]:
def plot_umap_by_group_labels_on(adata, color, title=None):
    plot = sc.pl.umap(adata, color=color, legend_loc='on data', title=title, show=False)
    return plot

def plot_umap_by_group_labels(adata, color, cmap_dict, title=None):
       # Plot the UMAP with the custom color map
    plot = sc.pl.umap(adata, color=color, groups=adata.obs.batch.unique(), palette=cmap_dict, legend_fontsize=12, show = False)
    return plot


In [20]:
def jaccard_index(set_a, set_b):
    """
    Calculate the Jaccard index between two sets.

    Args:
        set_a: A set of highly variable genes in dataset A.
        set_b: A set of highly variable genes in dataset B.

    Returns:
        A tuple containing the raw Jaccard index and the scaled Jaccard index (0 to 1).
    """
    intersection = set_a.intersection(set_b)
    union = set_a.union(set_b)

    # Calculate the Jaccard index
    jaccard = len(intersection) / len(union)

    return jaccard

In [21]:
from scipy.stats import hypergeom
from scipy.sparse import csr_matrix

def get_hvg_set(adata, n_top_genes=2000):
    adata_copy = adata.copy()
    if isinstance(adata_copy.X, csr_matrix):
        adata_copy.X = csr_matrix(adata_copy.X.maximum(0).toarray() + 1e-10)
    else:
        adata_copy.X = np.clip(adata_copy.X, 0, None)

    sc.pp.log1p(adata_copy)
    sc.pp.highly_variable_genes(adata_copy, n_top_genes=n_top_genes)
    hvg_set = set(adata_copy.var_names[adata_copy.var['highly_variable']])
    return hvg_set

In [22]:
import numpy as np
import scanpy as sc
from scipy.stats import hypergeom
from scipy.sparse import csr_matrix

def jaccard_index(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

def hvg_overlap_analysis(hvg_set_1, hvg_set_2, adata, dataset, n_top_genes):
    adata_copy = adata.copy()

    if isinstance(adata_copy.X, csr_matrix):
        adata_copy.X = dataset.inverse_transform(adata_copy.X.toarray())
    else:
        adata_copy.X = dataset.inverse_transform(adata_copy.X)

    if isinstance(adata_copy.X, csr_matrix):
        adata_copy.X = csr_matrix(adata_copy.X.maximum(0).toarray() + 1e-10)
    else:
        adata_copy.X = np.clip(adata_copy.X, 0, None)

    sc.pp.log1p(adata_copy)

    # Normalize and find HVGs for each dataset
    sc.pp.highly_variable_genes(adata_copy, n_top_genes=n_top_genes)

    # Calculate the overlap between the HVG sets
    hvg2 = set(adata_copy.var_names[adata_copy.var['highly_variable']])

    jaccard_train = jaccard_index(hvg_set_1, hvg2)
    jaccard_test = jaccard_index(hvg_set_2, hvg2)

    return jaccard_train, jaccard_test


In [23]:
def train_gan_one_epoch(epoch, GEX_dataloader_train, gan, g_opt, d_opt, criterion, z_dim):
    gan.generator.to(device)
    gan.discriminator.to(device)
    gan.train()
    total_loss_discriminator = 0
    total_loss_generator = 0
    for iteration, (data, _) in enumerate(GEX_dataloader_train):
        data = data.to(device)
        # Generate fake data
        noise = get_noise(data.shape[0], z_dim, device=device)
        fake_data = gan.generator(noise)

        fake_data = fake_data.to(device)

        # Train the discriminator
        d_opt.zero_grad()
        pred_real = gan.discriminator(data)
        pred_fake = gan.discriminator(fake_data.detach())
        loss_real = criterion(pred_real, torch.ones_like(pred_real))
        loss_fake = criterion(pred_fake, torch.zeros_like(pred_fake))
        loss_discriminator = (loss_real + loss_fake) / 2
        loss_discriminator.backward(retain_graph=True)
        wandb.log({"loss_discriminator": loss_discriminator})
        d_opt.step()

        # Train the generator
        g_opt.zero_grad()
        noise = get_noise(data.shape[0], z_dim, device=device)
        pred_fake = gan.discriminator(fake_data)
        loss_generator = criterion(pred_fake, torch.ones_like(pred_fake))
        loss_generator.backward()
        wandb.log({"loss_generator": loss_generator})
        g_opt.step()

        total_loss_discriminator += loss_discriminator.item()
        total_loss_generator += loss_generator.item()

    num_iterations = len(GEX_dataloader_train)
    mean_loss_discriminator = total_loss_discriminator / num_iterations
    mean_loss_generator = total_loss_generator / num_iterations

    return mean_loss_discriminator, mean_loss_generator



In [24]:
def train_gan_one_epoch_wasserstein(epoch, GEX_dataloader_train, gan, g_opt, d_opt, criterion, z_dim, clip_value, n_disc):
    gan.generator.to(device)
    gan.discriminator.to(device)
    gan.train()
    total_loss_discriminator = 0
    total_loss_generator = 0
    for iteration, (data, _) in enumerate(GEX_dataloader_train):    
        data = data.to(device)
        # Generate fake data
        noise = get_noise(data.shape[0], z_dim, device=device)
        fake_data = gan.generator(noise)

        fake_data = fake_data.to(device)

        # Train the discriminator
        d_opt.zero_grad()
        pred_real = gan.discriminator(data)
        loss_discriminator = torch.mean(gan.discriminator(fake_data)) - torch.mean(gan.discriminator(data))
        loss_discriminator.backward(retain_graph=True)
        wandb.log({"loss_discriminator": loss_discriminator})
        d_opt.step()
        # clip weights of discriminator
        for p in gan.discriminator.parameters():
          p.data.clamp_(-clip_value, clip_value)

        # Train the generator
        if iteration % n_disc == 0:
          g_opt.zero_grad()
          noise = get_noise(data.shape[0], z_dim, device=device)
          fake_data = gan.generator(noise)
          loss_generator = -torch.mean(gan.discriminator(fake_data))
          loss_generator.backward()
          wandb.log({"loss_generator": loss_generator})
          g_opt.step()

        total_loss_discriminator += loss_discriminator.item()
        total_loss_generator += loss_generator.item()

    num_iterations = len(GEX_dataloader_train)
    mean_loss_discriminator = total_loss_discriminator / num_iterations
    mean_loss_generator = total_loss_generator / num_iterations

    return mean_loss_discriminator, mean_loss_generator



In [25]:

def compute_gp(discriminator, real_data, fake_data, batch_size, device):
        eps = torch.rand(batch_size,1).to(device)
        # print(f"Batch size {batch_size} real data size {real_data.shape}, eps size {eps.shape}")
        eps = eps.expand_as(real_data).to(device)
        # Interpolation between real data and fake data.
        interpolation = eps * real_data + (1 - eps) * fake_data
        
        # get logits for interpolated images
        interp_logits = discriminator(interpolation)
        grad_outputs = torch.ones_like(interp_logits)
        
        # Compute Gradients
        gradients = torch.autograd.grad(
            outputs=interp_logits,
            inputs=interpolation,
            grad_outputs=grad_outputs,
            create_graph=True,
            retain_graph=True,
        )[0]
        
        # Compute and return Gradient Norm
        gradients = gradients.view(batch_size, -1)
        grad_norm = ((torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12) - 1) ** 2).mean() # Add manual calcualtion with epsilon term for stability (Derivatives of the gradient close to 0 can cause problems)
        # grad_norm = gradients.norm(2, 1)
        return torch.mean((grad_norm - 1) ** 2)

In [26]:
def train_gan_one_epoch_wasserstein_GP(epoch, GEX_dataloader_train, gan, g_opt, d_opt, loss_fn, z_dim, n_disc, w_gp, batch_size, device):
    gan.generator.to(device)
    gan.discriminator.to(device)
    gan.train()
    total_loss_discriminator = 0
    total_loss_generator = 0
    for iteration, (data, _) in enumerate(GEX_dataloader_train):    
        real_data = data.to(device)
        # Generate fake data
        noise = get_noise(batch_size, z_dim, device=device)
        fake_data = gan.generator(noise)


        fake_data = fake_data.to(device)


        # Train the discriminator
        d_opt.zero_grad()
        pred_real = gan.discriminator(real_data)

        gradient_penalty = w_gp * compute_gp(discriminator = gan.discriminator, real_data=real_data, fake_data=fake_data, batch_size=batch_size, device=device)

        loss_discriminator = torch.mean(gan.discriminator(fake_data)) - torch.mean(gan.discriminator(real_data))
        loss_discriminator = loss_discriminator + gradient_penalty

        loss_discriminator.backward(retain_graph=True)

        wandb.log({"loss_discriminator": loss_discriminator})
        wandb.log({"Gradient penaty": gradient_penalty})
        d_opt.step()

        # Train the generator
        if iteration % n_disc == 0:
          g_opt.zero_grad()
          noise = get_noise(batch_size, z_dim, device=device)
          fake_data = gan.generator(noise)
          loss_generator = -torch.mean(gan.discriminator(fake_data))
          loss_generator.backward()
          wandb.log({"loss_generator": loss_generator})
          g_opt.step()

        total_loss_discriminator += loss_discriminator.item()
        total_loss_generator += loss_generator.item()

    num_iterations = len(GEX_dataloader_train)
    mean_loss_discriminator = total_loss_discriminator / num_iterations
    mean_loss_generator = total_loss_generator / num_iterations

    return mean_loss_discriminator, mean_loss_generator


In [27]:
config = wandb.config = {
    "lr": 0.001,
    "batch_size": 1024,
    "epochs": 2000,
    "random_seed": 42,
    "dropout_G": 0,
    "dropout_D": 0.2,
    "split_criteria": "cell_type",
    "eval_size_percentage": 0.2,
    "hidden_sizes_G": [128, 256, 512],
    "hidden_sizes_D": [128,64, 32],
    "use_batch_norm_G": True,
    "use_batch_norm_D": True,
    "z_dim": 256,
    "clip_value" : 0.01,
    "n_disc" : 4,
    "w_gp" : 10,
    "device" : device 

}


In [28]:
print(scdata.obs["cell_type"].cat.categories.tolist())
print(scdata.obs["batch"].cat.categories.tolist())


['B1 B', 'CD4+ T activated', 'CD4+ T naive', 'CD8+ T', 'CD8+ T naive', 'CD14+ Mono', 'CD16+ Mono', 'Erythroblast', 'G/M prog', 'HSC', 'ID2-hi myeloid prog', 'ILC', 'Lymph prog', 'MK/E prog', 'NK', 'Naive CD20+ B', 'Normoblast', 'Plasma cell', 'Proerythroblast', 'Transitional B', 'cDC2', 'pDC']
['s1d1', 's1d2', 's1d3', 's2d1', 's2d4', 's2d5', 's3d3', 's3d6', 's3d7', 's3d10', 's4d1', 's4d8', 's4d9']


In [29]:
def train_func(config):
    run = wandb.init(project="single cell integration", entity="eamomc", config=config)

    config = wandb.config
    train, test = stratified_split(scdata, config.eval_size_percentage, config.random_seed, split_criteria=config.split_criteria)
    train_data = scdata[train]
    test_data = scdata[test]
    test_data_ = test_data.copy()
    test_data_ann = test_data
    wandb.log({"train_size": len(train_data), "test_size": len(test_data)})
    


    GEX_Dataset_train = GEX_Dataset(train_data, scaler="Standard", cat_var="batch", label_encoder="one_hot")
    GEX_dataloader_train = torch.utils.data.DataLoader(GEX_Dataset_train, batch_size=config.batch_size, shuffle=True, drop_last=True)

    GEX_Dataset_test = GEX_Dataset(test_data, scaler="Standard", cat_var="batch", label_encoder="one_hot")

    generator = Generator(output_size=GEX_Dataset_train.n_features, input_size=config.z_dim, hidden_sizes=config.hidden_sizes_G, dropout=config.dropout_G, use_batch_norm=config.use_batch_norm_G).to(config.device)
    discriminator = Discriminator(input_size=GEX_Dataset_train.n_features, output_size=1, hidden_sizes=config.hidden_sizes_D, use_batch_norm=config.use_batch_norm_D, dropout=config.dropout_D).to(config.device)

    gan = GAN(generator, discriminator).to(config.device)
    print(gan)

    loss_fn = nn.BCEWithLogitsLoss()
    d_opt = torch.optim.Adam(gan.discriminator.parameters(), lr=config.lr)
    g_opt = torch.optim.Adam(gan.generator.parameters(), lr=config.lr)

 

    data = np.array(GEX_Dataset_test[:])[0]
    test_data_ann.X = np.array(data)
    
    
    hyg_train = get_hvg_set(train_data, n_top_genes=2000)
    hvg_test =  get_hvg_set(test_data, n_top_genes=2000)

    # test_data_ = test_data
    # sc.pp.neighbors(test_data_ann, n_neighbors=10)
    # sc.tl.umap(test_data_ann)
    # test_UMAP_batch = sc.pl.umap(test_data_ann, show=False, color="batch")
    # test_UMAP_cell_type = sc.pl.umap(test_data_ann, show=False, color="cell_type")

    # wandb.log({"Test UMAP baseline batch": wandb.Image(test_UMAP_batch)})
    # wandb.log({"Test UMAP baseline cell type": wandb.Image(test_UMAP_cell_type)})

    unique_cell_types = umap_data.obs["cell_type"].unique().tolist()
    unique_batches = umap_data.obs["batch"].unique().tolist()
    # Define custom color dictionaries for cell types and batches
    custom_palette_cell_type = {cell_type: ('#00FF00' if cell_type == "fake" else None) for cell_type in unique_cell_types}
    custom_palette_batch = {batch: ('#00FF00' if batch == "fake" else None) for batch in unique_batches}


    gan.train()
    for epoch in tqdm(range(1, config.epochs + 1)):
        wandb.log({"epoch": epoch})
        D_loss_train, G_loss_train = train_gan_one_epoch_wasserstein_GP(epoch=epoch, gan=gan, GEX_dataloader_train=GEX_dataloader_train, d_opt=d_opt, g_opt=g_opt, loss_fn=loss_fn, z_dim=config.z_dim, batch_size=config.batch_size, device=config.device, n_disc=config.n_disc, w_gp=config.w_gp)
        wandb.log({"D_loss_train": D_loss_train, "G_loss_train": G_loss_train})

    eval_condition = (
          (epoch <= 10 and epoch % 2 == 0) or
          (10 < epoch <= 100 and epoch % 25 == 0) or
          (100 < epoch <= 1000 and epoch % 100 == 0) or
          (1000 < epoch <= 2000 and epoch % 200 == 0)
      )




    if eval_condition:
      umap_data = generate_fake_data_and_combine(test_data_ann, generator, config.z_dim, config.batch_size)
      umap_data_fake = umap_data[umap_data.obs["group"] == "fake"].copy()
      jaccard_train, jaccard_test = hvg_overlap_analysis(hyg_train, hvg_test, umap_data_fake, GEX_Dataset_train, n_top_genes=2000)
      wandb.log({"Jaccard Train": jaccard_train, "Jaccard Test": jaccard_test})

      umap_real_and_fake = sc.pl.umap(umap_data, color="group", show=False, title=f"UMAP Real and Fake (Epoch {epoch})", palette=custom_palette_cell_type)
      umap_batch = sc.pl.umap(umap_data, color="batch", show=False, title=f"UMAP Batch (Epoch {epoch})", palette=custom_palette_batch)
      umap_cell_type = sc.pl.umap(umap_data, color="cell_type", show=False, title=f"UMAP Cell Type (Epoch {epoch})", palette=custom_palette_cell_type)


    
      # Save UMAP images to temporary files
      with tempfile.NamedTemporaryFile(suffix=".png") as tmp_umap_real_and_fake:
          umap_real_and_fake.figure.savefig(tmp_umap_real_and_fake.name, dpi=300, bbox_inches='tight')
          wandb.log({"UMAP Real and Fake": wandb.Image(tmp_umap_real_and_fake.name)})

      with tempfile.NamedTemporaryFile(suffix=".png") as tmp_umap_batch:
          umap_batch.figure.savefig(tmp_umap_batch.name, dpi=300, bbox_inches='tight')
          wandb.log({"UMAP Batch": wandb.Image(tmp_umap_batch.name)})

      with tempfile.NamedTemporaryFile(suffix=".png") as tmp_umap_cell_type:
          umap_cell_type.figure.savefig(tmp_umap_cell_type.name, dpi=300, bbox_inches='tight')
          wandb.log({"UMAP Cell Type": wandb.Image(tmp_umap_cell_type.name)})

      
      if epoch > 100:
        generator_weights_path = os.path.join(wandb.run.dir, f"generator_epoch_{epoch}.pt")
        discriminator_weights_path = os.path.join(wandb.run.dir, f"discriminator_epoch_{epoch}.pt")
        torch.save(gan.generator.state_dict(), generator_weights_path)
        torch.save(gan.discriminator.state_dict(), discriminator_weights_path)

        # Log the model weights to wandb
        wandb.save(generator_weights_path)
        wandb.save(discriminator_weights_path)



     

       


In [ ]:

train_func(config=config)

wandb: Currently logged in as: eamomc. Use `wandb login --relogin` to force relogin


GAN(
  (generator): Generator(
    (model): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=128, out_features=256, bias=True)
      (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Linear(in_features=256, out_features=512, bias=True)
      (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
      (9): Linear(in_features=512, out_features=13431, bias=True)
    )
  )
  (discriminator): Discriminator(
    (model): Sequential(
      (0): Linear(in_features=13431, out_features=128, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
      (2): Dropout(p=0.2, inplace=False)
      (3): Linear(in_features=128, out_features=64, bias=True)
      (4): InstanceNorm1d(64, eps=1e-05, momentum=0.1, affine=False, track_r

<ipython-input-29-e03457f0f497>:31: FutureWarning:

The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.

<ipython-input-29-e03457f0f497>:31: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/usr/local/lib/python3.9/dist-packages/scipy/sparse/_index.py:137: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
config = wandb.config


In [ ]:
config = wandb.config
train, test = stratified_split(scdata, config.eval_size_percentage, config.random_seed, split_criteria=config.split_criteria)
train_data = scdata[train]
test_data = scdata[test]
test_data_ = test_data

In [ ]:
test_data.X.toarray()

In [ ]:
train_data.X.toarray()